In [1]:
import pandas as pd
import numpy as np

In [4]:
book_counts = pd.read_csv('../data/book_review_count.csv',index_col=0)
user_counts = pd.read_csv('../data/user_review_count.csv',index_col=0)
valid_df = pd.read_csv('../valid/valid_FM.csv')

In [41]:
def cold_users(n: int=1)->set():
    cold_users = set(user_counts[user_counts['count']<=n]['user_id'])
    return cold_users

def cold_books(n:int=0)->set():
    cold_books = set(book_counts[book_counts['count']<=n]['isbn'])
    return cold_books

def df_rmse(df:pd.DataFrame):
    return ((df.target - df.rating)**2).mean()**.5 , len(df)

def conditional_rmse(df:pd.DataFrame, userset:set={}, bookset:set={}):
    is_colduser = df['user_id'].isin(userset)
    is_coldbook = df['isbn'].isin(bookset)
    
    print("-*"*10+"두가지로 나눈 경우"+"-*"*10)
    print("----------유저 기준으로 나눴을 때----------")
    error,length = df_rmse(df[is_colduser])
    print(f'RMSE = {error}, {length}개 - 유저 COLD')
    error,length = df_rmse(df[~is_colduser])
    print(f'RMSE = {error}, {length}개 - 유저 데이터가 {n+1}개 이상인 경우')
    print()

    print("----------책 기준으로 나눴을 때----------")
    error,length = df_rmse(df[is_coldbook])
    print(f'RMSE = {error}, {length}개 - 책 COLD')
    error,length = df_rmse(df[~is_coldbook])
    print(f'RMSE = {error}, {length}개 - 책 데이터가 {n+1}개 이상인 경우')
    print()

    print("----------유저,책 둘 기준으로 나눴을 때----------")
    error,length = df_rmse(df[is_colduser&is_coldbook])
    print(f'RMSE = {error}, {length}개 - 유저 COLD, 책 COLD')
    error,length = df_rmse(df[~(is_colduser&is_coldbook)])
    print(f'RMSE = {error}, {length}개 - 유저와 책 둘 다 데이터가 {n+1}개 이상인 경우')
    print('\n')
    
    print("-*"*10+"네가지로 나눈 경우"+"-*"*10)
    error,length = df_rmse(df[is_colduser&is_coldbook])
    print(f'RMSE = {error}, {length}개 - 유저 COLD, 책 COLD')
    print()
    error,length = df_rmse(df[is_colduser&~is_coldbook])
    print(f'RMSE = {error}, {length}개 - 유저 COLD, 책 HOT')
    print()
    error,length = df_rmse(df[~is_colduser&is_coldbook])
    print(f'RMSE = {error}, {length}개 - 유저 HOT, 책 COLD')
    print()
    error,length = df_rmse(df[~is_colduser&~is_coldbook])
    print(f'RMSE = {error}, {length}개 - 유저 HOT, 책 HOT')
    print()

    print("-"*20)
    error,length = df_rmse(df)
    print(f'RMSE = {error}, {length}개 - 총 데이터')


In [42]:
n = 10
cu_n = cold_users(n)
cb_n = cold_books(n)
print(f"Cold Start {n}개 기준")
conditional_rmse(valid_df,cu_n,cb_n)

Cold Start 10개 기준
-*-*-*-*-*-*-*-*-*-*두가지로 나눈 경우-*-*-*-*-*-*-*-*-*-*
----------유저 기준으로 나눴을 때----------
RMSE = 2.528396104209113, 22369개 - 유저 COLD
RMSE = 2.2207112530439166, 38990개 - 유저 데이터가 11개 이상인 경우

----------책 기준으로 나눴을 때----------
RMSE = 2.397914950695296, 42466개 - 책 COLD
RMSE = 2.1959092953544728, 18893개 - 책 데이터가 11개 이상인 경우

----------유저,책 둘 기준으로 나눴을 때----------
RMSE = 2.621382532056494, 14122개 - 유저 COLD, 책 COLD
RMSE = 2.2457762328529385, 47237개 - 유저와 책 둘 다 데이터가 11개 이상인 경우


-*-*-*-*-*-*-*-*-*-*네가지로 나눈 경우-*-*-*-*-*-*-*-*-*-*
RMSE = 2.621382532056494, 14122개 - 유저 COLD, 책 COLD

RMSE = 2.360677290813678, 8247개 - 유저 COLD, 책 HOT

RMSE = 2.278409448364003, 28344개 - 유저 HOT, 책 COLD

RMSE = 2.0592282098283383, 10646개 - 유저 HOT, 책 HOT

--------------------
RMSE = 2.337576310905986, 61359개 - 총 데이터
